In [4]:
import os
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from elevenlabs import voices, generate, play, save
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

eleven_api_key = "34dbe78c2a8c6642fcdc27dbb44c1929"

In [11]:
def podify(topic, style, num_minutes, voice):


    num_words = num_minutes*135 # Approximately 135 words per minute with TTS voice (refine this)

    audio_voice = voice.split()[0]
    summary_lang = "English"
    if voice.split()[1] == "(SWE)":
        summary_lang = "Swedish"
    if voice.split()[1] == "(IT)":
        summary_lang = "Italian"

    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    text = wikipedia.run(topic)    

    return "audiofiles/news_eli5.wav"

    # Connect to OpenAI, prompt ChatGPT to summarize the text
    chat_model = ChatOpenAI() # Must have set API key as env var
    prompt = "Summarize the following text in {} words. {}. The summary must be in {}. Here's the text: {}".format(num_words, style, summary_lang, text) 
    summary = chat_model.predict(prompt)

    # Connect to elevenlabs, generate audio
    audio = generate(text=summary, voice=audio_voice, model="eleven_monolingual_v1", api_key=eleven_api_key)

    # Save audiofile
    if not os.path.exists("audiofiles"):
        os.makedirs("audiofiles")

    # Handling identical file names
    file_suffix = 0
    while os.path.exists("audiofiles/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix)):
        file_suffix +=1

    save(audio, "audiofiles/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix))

    return "audiofiles/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix)




In [12]:
demo = gr.Interface(fn=podify, 
                    inputs=[
                            gr.Textbox(label="What do you want your podcast to be about?", placeholder="E.g. 'This week's news in AI'"),
                            gr.Textbox(label="Do you want to customize the style of your podcast?", placeholder="E.g. 'Explain this to me like I'm five years old'"),
                            gr.Slider(minimum=0.5, maximum=5, step=0.5, label="Length", info = "Number of minutes of audio summary"),
                            gr.Dropdown(["Adam (US)", "Antoni (IT)", "Arnold (US)", "Bella (US)", "Domi (US)", "Elli (US)", "Josh (SWE)", "Rachel (US)", "Sam (US)"], label="Voice", info="Narrator voice and language of podcast"),
                            ],
                    outputs= ["audio"]
                    )
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://1543c61be437f5630b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/langchain/utilities/wikipedia.py", line 33, in validate_environment
    import wikipedia
ModuleNotFoundError: No module named 'wikipedia'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/gradio/routes.py", line 544, in predict
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/site-packages/gradio/route_utils.py", line 217, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/site-packages/gradio/blocks.py", line 1553, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/site-packages/gradio/blocks.py", line 1191, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/site-packages/anyio/to_thread.py", line 33, in run_sync
    return await get

def podify(topic, style, num_minutes, voice, env="prod"):
    num_words = num_minutes*135 # Approximately 135 words per minute with TTS voice (refine this)

    audio_voice = voice.split()[0]
    summary_lang = "English"
    if voice.split()[1] == "(SWE)":
        summary_lang = "Swedish"
    if voice.split()[1] == "(IT)":
        summary_lang = "Italian"

    article_titles = list()
    article_body = list()
    if(env == "prod"):
        google_news = gnews.GNews(period='3d', max_results=3)
        news_response = google_news.get_news(topic) # En lista av artikelobjekt, article.text, article.title

        for news in news_response:
            full_article = google_news.get_full_article(news['url'])
            article_titles.append(full_article.title)
            article_body.append(full_article.text)
        with open('./tmp/articles.txt', 'w') as f:   
            for i in range(len(article_titles)):
                if(i != 0):
                    f.write(f"********\n")
                f.write(f"{article_titles[i]}\n")
                f.write(f"********\n")
                f.write(f"{article_body[i]}\n")
    elif(env == "test"):
        with open('./tmp/articles.txt') as f:
            lines = f.readlines()
            lines = ''.join([line.strip() for line in lines])
            lines = lines.split("********")
            for i in range(len(lines)):
                line = lines[i]
                if(i % 2 == 0):
                    article_titles.append(line)
                else:
                    article_body.append(line)
    # print(article_titles)
    # print(article_body)
    combined_text = ""
    for article in article_body:
        combined_text += article

    # Get segments from txt by splitting on .
    segments =  combined_text.split('.')
    # Put the . back in
    segments = [segment + '.' for segment in segments]
    # Further split by comma
    segments = [segment.split(',') for segment in segments]
    # Flatten
    segments = [item for sublist in segments for item in sublist]

    print(segments)
    return
    ### TASK 2 ###
    ### Här borde man använda ChatGPT för att gå fråga om artiklarna är relevanta för the user topic ###
    ### Hämta alla articles.title, fråga om "Is this (title) / (first paragraph) relevant to answer (user topic)"
    ### Behöver inte göra en i taget, skicka en lista av titlar/paragrafer i en request - fråga vilka som är relavantas
    

    # chat_model = ChatOpenAI(model_name='gpt-3.5-turbo') # Must have set API key as env var
    # print(article_titles)
    # prompt = f'A listener wants to learn about {topic}. This is a list of articles we will give to the listener. {article_titles}. Are these articles relevant based on the title? Return only the titles of all articles that are NOT relevant in the excat same woridng'
    # # prompt = "Summarize the following text in {} words. {}. The summary must be in {}. Here's the text: {}".format(num_words, style, summary_lang, all_news_text) # The text contains several news articles, separated by this sign '****'.
    # test = chat_model.predict(prompt)
    # print(test)
       

    ### TASK 3: Nu har jag bara en jävligt lång kontext-version av GPT, men här vill man köra summeringstekniker av all_news_text
    # Connect to OpenAI, prompt ChatGPT to summarize the text
    chat_model = ChatOpenAI(model_name='gpt-3.5-turbo-16k') # Must have set API key as env var
    prompt = "Summarize the following text in {} words. {}. The summary must be in {}. Here's the text: {}".format(num_words, style, summary_lang, all_news_text) # The text contains several news articles, separated by this sign '****'.
    summary = chat_model.predict(prompt)

    # Connect to elevenlabs, generate audio
    audio = generate(text=summary, voice=audio_voice, model="eleven_monolingual_v1", api_key=eleven_api_key)
    intro_audio = generate(text='You are listening to an AI-generated podcast on the theme {}, provided to you by PerfectPod AI'.format(topic), voice=audio_voice, model="eleven_monolingual_v1", api_key=eleven_api_key)

    # TASK 4, gör detta bättre. Kom på sätt att göra detta effektivt. Lite onödigt kanske med två olika genereringar av ljud? Hur kan man lägga in ljudeffekter bra? (addrrar ljudfiler, overlayar ljud)

    # Save audiofile
    if not os.path.exists("news_audio"):
        os.makedirs("news_audio")

    # Handling identical file names
    file_suffix = 0
    while os.path.exists("news_audio/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix)):
        file_suffix +=1

    save(audio, "news_audio/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix))
    save(intro_audio, "news_audio/summary_in_{}_mins_{}_INTRO.wav".format(num_minutes, file_suffix)) 

    concatenate_audio_moviepy(["news_audio/summary_in_{}_mins_{}_INTRO.wav".format(num_minutes, file_suffix), "sound_effects/original_transition.wav"], "news_audio/summary_in_{}_mins_{}_INTERMED.wav".format(num_minutes, file_suffix))
    concatenate_audio_moviepy(["news_audio/summary_in_{}_mins_{}_INTERMED.wav".format(num_minutes, file_suffix), "news_audio/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix)], "news_audio/summary_in_{}_mins_{}_READY.wav".format(num_minutes, file_suffix))

    return "news_audio/summary_in_{}_mins_{}_READY.wav".format(num_minutes, file_suffix)


    ### TASK 5: Hur får vi långa poddar. Vi vill ha lång output, och vi måste experimentera med detta.
    ### Se hur folk generar hela böcker med ChatGPT, medium. Ex generera templates, generera rekursivt delar. Ha med summeringar av tidigare delar?


podify("Donald Trump", "ELI5", 1, "Bella (US)", "test")